In [0]:
%sql
--Sales
-- Salesorderdetail --> SalesOrderID <======> salesorderheader --> SalesOrderID
-- personbronze --> BusinessentityID <======> BusinessEntitybronze --> BusinessentityID
-- businessentityaddress_bronze --> addressID <======> address_bronze --> addressID
-- businessentityaddress_bronze --> addresstypeID <======> addresstype_bronze --> addresstypeID
-- Product Product_bronze --> ProductModelID <=====> ProductModelID --> productmodel_bronze


-- culture_bronze..???

In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
salesorderdetail_df = spark.sql("select * from training.piyush.salesorderdetail_bronze")
salesorderheader_df = spark.sql("select * from training.piyush.salesorderheader_bronze")
salesorderdetail_df.printSchema()
salesorderheader_df.printSchema()

**Approach 1**

In [0]:
# desired_schema = {'ModifiedDate': 'yyyy-MM-dd', '_rescued_data': 'yyyy-MM-dd', 'year': 'yyyy'}
# def Sales_casting(df):
#        for column, dtype in desired_schema.items():
#               print(column, dtype)
#               df = df.withColumn(column, to_date(df[column], dtype))
# #     df = df.withColumn("ModifiedDate", to_date(df.ModifiedDate, 'yyyy-MM-dd'))
#        #     .withColumn('_rescued_data', to_date(df._rescued_data, 'yyyy-MM-dd'))
#        return df

In [0]:
vardf = Sales_casting(salesorderdetail_df)

**Approach 2**

In [0]:
def infer_schema_type(salesorderdetail_df, col_name, sample_data = 500):

    sample_df = salesorderdetail_df.select(col_name).limit(sample_data)

    if sample_df.filter(col(col_name).cast("int").isNotNull()).count() == sample_df.count():
        return 'int'
    if sample_df.filter(col(col_name).cast("double").isNotNull()).count() == sample_df.count():
        return 'double'
    if sample_df.filter(to_date(col(col_name), 'yyyy-MM-dd').isNotNull()).count() == sample_df.count():
        return 'date'
    
    return 'string'

In [0]:
def infer_schema(salesorderdetail_df):
    inferred = {}
    for field in salesorderdetail_df.schema.fields:
        col_name = field.name
        # print(col_name)
        inferred[col_name] = infer_schema_type(salesorderdetail_df, col_name)
    return inferred


In [0]:
schema_mapped = infer_schema(salesorderdetail_df)
print(schema_mapped)

for c, dtype in schema_mapped.items():
    # print(c, dtype)
    salesorderdetail_df = salesorderdetail_df.withColumn(c, col(c).cast(dtype))

salesorderdetail_df.printSchema()

**Approach 3**

In [0]:
def infer_type(col_name, df):
    current_type = df.schema[col_name].dataType
    if current_type == "timestamp":
        return "timestamp"
    if col_name in ['Ingested_time']:
        return "timestamp"
    elif col_name in ["ModifiedDate"]:
        return "date"
    else:
        # existing logic to infer data type
        df = df.withColumn("test_int", col(col_name).cast("int")) \
               .withColumn("test_double", col(col_name).cast("double")) \
               .withColumn("test_ts", col(col_name).cast("date"))
 
        int_count = df.filter(col("test_int").isNotNull()).count()
        double_count = df.filter(col("test_double").isNotNull()).count()
        ts_count = df.filter(col("test_ts").isNotNull()).count()
 
        total = df.count()
        if int_count == total:
            return "int"
        elif double_count == total:
            return "double"
        elif ts_count == total:
            return "timestamp"
        else:
            return "string"

In [0]:
def auto_cast_df(df):
    for col_name in df.columns:
        inferred_type = infer_type(col_name, df.select(col_name).limit(1000))
        df = df.withColumn(col_name, col(col_name).cast(inferred_type))
    return df

In [0]:
bronze_tables = ['address_bronze', 'addresstype_bronze', 'businessentityaddress_bronze']
# , 'culture_bronze', 'customer_bronze', 'emailaddress_bronze', 'person_bronze', 'product_bronze', #'productcategory_bronze', 'productdescription_bronze', 'productmodel_bronze', #'productmodelproductdescriptionculture_bronze','productsubcategory_bronze', 'salesorderdetail_bronze', #'salesorderheader_bronze']

def silver_auto_casted():
    for table_name in bronze_tables:
            bronze_df = spark.read.table(f"training.piyush.{table_name}")
    return auto_cast_df(bronze_df)

In [0]:
silver_df= silver_auto_casted()
display(silver_df.printSchema())

In [0]:
silver_df.display()

In [0]:
salesorderheader_df.schema.fields

In [0]:
distinct_count = salesorderdetail_df.select("SalesOrderID").distinct().count()
print(distinct_count)
# -- print(distinct_count)

In [0]:
%sql
select count(*) from training.piyush.product_silver

In [0]:
address_bronzedf = spark.sql("select * from training.piyush.addresstype_bronze").printSchema()
address_SIlevrdf = spark.sql("select * from training.piyush.addresstype_silver").printSchema()

In [0]:
df = dlt.read_stream('training.piyush.address_bronze')

In [0]:
%sql
select count(*) from training.piyush.product_bronze where ProductID is null


In [0]:
quality_checks = {
    "address": {
        "AddressID": "AddressID IS NOT NULL",
        "ModifiedDate": "ModifiedDate IS NOT NULL"
    },
    "customer": {
        "CustomerID": "CustomerID IS NOT NULL"
    },
    "addresstype":{
        "AddressTypeID": "AddressTypeID IS NOT NULL"
   },
    "emailaddress":{
        "EmailAddressID": "EmailAddressID IS NOT NULL"
    },
    "product":{
        "ProductID": "ProductID IS NOT NULL"
    },
    "person":{
        "BusinessEntityID": "BusinessEntityID IS NOT NULL"
    }
}

In [0]:
# validation_expr = " AND ".join(
#         [quality_checks[field] for field in quality_checks.keys() if field in df.columns]
#     )

In [0]:
# for field in quality_checks.values():
#     print(field)
#     for col, condition in field.items():
#         print(col, condition)

# # validation_expr = " AND ".join([condition for table in quality_checks.values() for column, condition in table.items()])

# print(validation_expr)
for x in quality_checks:
    print(quality_checks["address"])
    print(x.values())

    